In [1]:
from z3 import *
print(z3.get_version()) 

(4, 14, 1, 0)


In [2]:
# 阶段
Stage, stage_consts = EnumSort('Stage', ['PREPARE', 'CONFIRM', 'EXTERNALIZE'])
# 候选值
Cand = IntSort()
# ballot
Ballot_Nil, ballot_nil = EnumSort('Ballot_Nil', ['ballot_nil'])
ballot_nil = ballot_nil[0] #nothing in list

Ballot = Datatype('Ballot')
Ballot.declare('cons', ('n', IntSort()), ('x', Cand))
Ballot.declare('nil')
Ballot = Ballot.create()
ballot_nil = Ballot.nil



# 节点状态 记录phi b p' p c h z 暂时不记录M
Nstate = Datatype('Nstate')
Nstate.declare('cons', 
    ('phi', Stage),   # 阶段字段
    ('b', Ballot),    # ballot 字段
    ('p1', Ballot), 
    ('p', Ballot), 
    ('c', Ballot), 
    ('h', Ballot), 
    ('z', IntSort())
)
Nstate = Nstate.create()  


phi = Nstate.phi  #phase
b = Nstate.b      #ballot
p1 = Nstate.p1    #prepared
p = Nstate.p        #prepared
c = Nstate.c    #commit
h = Nstate.h    #highest prepared
z = Nstate.z
# 时间步长
T = 100
times = [f'node{i}' for i in range(T)]
Time, time_consts = EnumSort('Time', times)
times =time_consts

In [3]:
n = 3  # 节点个数
m = 1  # 消息个数

# 定义枚举类型
nodes = [f'node{i+1}' for i in range(n)]
Node, node_consts = EnumSort('Node', nodes)
nodes = node_consts  # 直接使用常量列表

ballots = [Ballot.cons(1, 1)]
# 节点集合的位向量表示
nsets = [BitVecVal(i, n) for i in range(2**n)]

# 集合成员检查（添加 time 参数）
in_set = Function('in_set', Node, BitVecSort(n), Time, BoolSort())
well_behave = Function('well_behave', Node, Time, BoolSort())

# 信任分片和阻塞点（添加 time 参数）
in_qslice = Function('in_qslice', Node, Node, Time, BoolSort())
is_blocking_set = Function('is_blocking_set', Node, BitVecSort(n), Time, BoolSort())
is_quorum = Function('is_quorum', BitVecSort(n), Time, BoolSort())
is_blocking_point = Function('is_blocking_point', Node, Node, Time, BoolSort())

# 有关恶意节点
is_dset_c = Function('is_dset_c', BitVecSort(n),Time, BoolSort()) # dset 补 在带量词的形式化中如此实现较为方便，所以此处同意
is_intact = Function('is_intact', Node, Time,BoolSort()) # 是否为完整节点
is_quorum_within = Function('is_quorum_within', BitVecSort(n), Time,BitVecSort(n), BoolSort()) # 第一个集合在第二个集合中为quorum

# 投票协议相关谓词（全部添加 time 参数）
init_prepared = Function('init_prepared', Node, IntSort(), Time, BoolSort())  # Prepared 替换为 IntSort()
prepared = Function('prepared', Node, IntSort(), Time, BoolSort())  # Prepared 替换为 IntSort()
ratify_prepared = Function('ratify_prepared', Node, IntSort(), Time, BoolSort())  # Prepared 替换为 IntSort()
accept_prepared = Function('accept_prepared', Node, Ballot, Time, BoolSort())  # Prepared 替换为 IntSort()
confirm_prepared = Function('confirm_prepared', Node, Ballot, Time, BoolSort())  # Prepared 替换为 IntSort()
local_vote_prepared = Function('local_vote_prepared', Node, Node, IntSort(), Time, BoolSort())  # Prepared 替换为 IntSort()
local_quorum_prepared_ratify = Function('local_quorum_prepared_ratify', Node, BitVecSort(n), IntSort(), Time, BoolSort())  # Prepared 替换为 IntSort()
local_ratify_prepared = Function('local_ratify_prepared', Node, Node, IntSort(), Time, BoolSort())  # Prepared 替换为 IntSort()
local_accept_prepared = Function('local_accept_prepared', Node, Node, IntSort(), Time, BoolSort())  # Prepared 替换为 IntSort()

In [4]:
node_state = Function('node_state', Node, Time, Nstate) # 每个节点的状态

In [5]:
# 定义节点量化变量（用于ForAll）
node1 = Const('node1', Node)

time = Const('time', Time)



In [6]:
# 正确访问字段
phi_value = phi(node_state(node1, time))  # 使用 phi() 函数访问字段
print(phi_value)

phi(node_state(node1, time))


In [7]:
PREPARE, CONFIRM, EXTERNALIZE = stage_consts
# 初始化约束（统一使用解包后的枚举常量）
Init = [
    [ phi(node_state(nodes[i1], time)) == PREPARE for i1 in range(n) ],
    [ b(node_state(nodes[i1], time)) == ballot_nil for i1 in range(n) ],
    [ z(node_state(nodes[i1], time)) == 0 for i1 in range(n) ],
    [ p1(node_state(nodes[i1], time)) == ballot_nil for i1 in range(n) ],
    [ p(node_state(nodes[i1], time)) == ballot_nil for i1 in range(n) ],
    [ c(node_state(nodes[i1], time)) == ballot_nil for i1 in range(n) ],
    [ h(node_state(nodes[i1], time)) == ballot_nil for i1 in range(n) ]
]


In [8]:
# 展平约束
def flatten(l):
    return [s for t in l for s in t]

In [9]:
s = Solver()
s.add(flatten(Init))

# 验证所有节点的初始阶段为 PREPARE
s.add(phi(node_state(node1, time)) != PREPARE)
if s.check() == unsat:
    print("初始化约束有效：所有节点初始阶段为 PREPARE")
else:
    print("约束存在漏洞！")

初始化约束有效：所有节点初始阶段为 PREPARE


In [10]:
Defi2 = [
    # (1) is_quorum(nsets[j], time)
    [
        And(
            And([
                Implies(
                    Extract(i1, i1, nsets[j]) == 1,
                    And([
                        Implies(
                            in_qslice(nodes[i1], nodes[i2], time),
                            Extract(i2, i2, nsets[j]) == 1
                        )
                        for i2 in range(n)
                    ])
                )
                for i1 in range(n)
            ]),
            Not(nsets[j] == 0)
        )
        == is_quorum(nsets[j], time)
        for j in range(2**n)
    ],
    
    # (2) in_qslice(...) == is_blocking_point(...)
    [
        in_qslice(nodes[i1], nodes[i2], time)
        == is_blocking_point(nodes[i1], nodes[i2], time)
        for i1 in range(n) for i2 in range(n)
    ],

    # (3) is_quorum_within(nsets[j], time, nsets[j1])
    [
        And(
            And([
                Implies(
                    Extract(i1, i1, nsets[j]) == 1,
                    And(
                        Extract(i1, i1, nsets[j1]) == 1,
                        And([
                            Implies(
                                And(
                                    Extract(i2, i2, nsets[j1]) == 1,
                                    in_qslice(nodes[i1], nodes[i2], time)
                                ),
                                Extract(i2, i2, nsets[j]) == 1
                            )
                            for i2 in range(n)
                        ])
                    )
                )
                for i1 in range(n)
            ]),
            Not(nsets[j] == 0)
        )
        == is_quorum_within(nsets[j], time, nsets[j1])
        for j in range(2**n)
        for j1 in range(2**n)
    ],

    
    # (4) is_dset_c(nsets[j], time)
    [
        And(
            And([
                Implies(
                    And(
                        is_quorum_within(nsets[j1], time, nsets[j]),
                        is_quorum_within(nsets[j2], time, nsets[j])
                    ),
                    Not(nsets[j1] & nsets[j2] == 0)
                )
                for j1 in range(2**n)
                for j2 in range(2**n)
            ]),
            Or(is_quorum(nsets[j], time), nsets[j] == 0)
        )
        == is_dset_c(nsets[j], time)
        for j in range(2**n)
    ],

    # (5) is_intact(nodes[i1], time)
    [
        Or([
            And(
                is_dset_c(nsets[j], time),
                Extract(i1, i1, nsets[j]) == 1,
                And([
                    Implies(
                        Not(well_behave(nodes[i2], time)),
                        Extract(i2, i2, nsets[j]) == 0
                    )
                    for i2 in range(n)
                ])
            )
            for j in range(2**n)
        ])
        == is_intact(nodes[i1], time)
        for i1 in range(n)
    ],
]


In [11]:
# 修改后的 C1
C1 = [
    # 规则 1：每个节点的 qslice 包含自身
    [Implies(well_behave(nodes[i1],time), in_qslice(nodes[i1], nodes[i1],time)) for i1 in range(n)],
    
    # 规则 2：两个 quorum 的交集非空
    [Implies(
        And(is_quorum(nsets[j1], time), is_quorum(nsets[j2], time)),
        Or([
            And(Extract(i1, i1, nsets[j1]) == 1, Extract(i1, i1, nsets[j2]) == 1)
            for i1 in range(n)
        ])
    ) for j1 in range(2**n) for j2 in range(2**n)],

    # 规则 3：至少有一个节点对每个候选值进行了 init_prepared
    [Or([init_prepared(nodes[i1], x, time) for i1 in range(n)]) for x in range(m)]  # 替换 prepareds 为 range(m)
]



In [12]:
C2 = [

    # (1) 原: Implies(well_behave(nodes[i1]), Or(init_nomi(nodes[i1], x), nomi(nodes[i1], x)) == local_vote_nomi(nodes[i2], nodes[i1], x))
    #     改: well_behave(..., time), init_prepared(..., time), prepared(..., time), local_vote_prepared(..., time)
    [
        Implies(
            well_behave(nodes[i1], time),
            Or(
                init_prepared(nodes[i1], x, time),
                prepared(nodes[i1], x, time)
            ) == local_vote_prepared(nodes[i2], nodes[i1], x, time)
        )
        for i1 in range(n)
        for i2 in range(n)
        for x in range(m)
    ],

    # (2) 原: Or([And(in_qslice(nodes[i2], nodes[i1]), local_vote_nomi(nodes[i2], nodes[i1], x)) for i1 in range(n)]) == nomi(nodes[i2], x)
    #     改: in_qslice(..., time), local_vote_prepared(..., time), prepared(..., time)
    [
        Or([
            And(
                in_qslice(nodes[i2], nodes[i1], time),
                local_vote_prepared(nodes[i2], nodes[i1], x, time)
            )
            for i1 in range(n)
        ]) == prepared(nodes[i2], x, time)
        for i2 in range(n)
        for x in range(m)
    ],

    # (3) 原: And(And([... local_vote_nomi(nodes[i1], nodes[i2], x) ...]), is_quorum(nsets[j])) == local_quorum_ratify(nodes[i1], nsets[j], x)
    #     改: local_vote_prepared(..., time), is_quorum(..., time), local_quorum_prepared_ratify(..., time)
    [
        And(
            And([
                Implies(
                    Extract(i2, i2, nsets[j]) == 1,
                    local_vote_prepared(nodes[i1], nodes[i2], x, time)
                )
                for i2 in range(n)
            ]),
            is_quorum(nsets[j], time)
        ) == local_quorum_prepared_ratify(nodes[i1], nsets[j], x, time)
        for i1 in range(n)
        for j in range(2**n)
        for x in range(m)
    ],

    # (4) 原: Or([And(Extract(i2, i2, nsets[j]) == 1, local_quorum_ratify(nodes[i1], nsets[j], x)) for j in range(2**n)]) == local_ratify_nomi(nodes[i1], nodes[i2], x)
    #     改: local_quorum_prepared_ratify(..., time), local_ratify_prepared(..., time)
    [
        Or([
            And(
                Extract(i2, i2, nsets[j]) == 1,
                local_quorum_prepared_ratify(nodes[i1], nsets[j], x, time)
            )
            for j in range(2**n)
        ]) == local_ratify_prepared(nodes[i1], nodes[i2], x, time)
        for i1 in range(n)
        for i2 in range(n)
        for x in range(m)
    ],

    # (5) 原: Or(Or([And(Extract(i1, i1, nsets[j]) == 1, is_quorum(nsets[j]), local_quorum_ratify(...))]), Or([And(is_blocking_point(...), local_ratify_nomi(...))])) == accept(nodes[i1], x)
    #     改: is_quorum(..., time), local_quorum_prepared_ratify(..., time), is_blocking_point(..., time), local_ratify_prepared(..., time), accept_prepared(..., time)
    [
        Or(
            Or([
                And(
                    Extract(i1, i1, nsets[j]) == 1,
                    is_quorum(nsets[j], time),
                    local_quorum_prepared_ratify(nodes[i1], nsets[j], x, time)
                )
                for j in range(2**n)
            ]),
            Or([
                And(
                    is_blocking_point(nodes[i1], nodes[i2], time),
                    local_ratify_prepared(nodes[i1], nodes[i2], x, time)
                )
                for i2 in range(n)
            ])
        ) == accept_prepared(nodes[i1], Ballot.cons(x, m), time)
        for i1 in range(n)
        for x in range(m)
    ],
    # [Or(Or([And(Extract(i1, i1, nsets[j]) == 1, is_quorum(nsets[j],time), local_quorum_prepared_ratify(nodes[i1], nsets[j], x,time)) for j in range(2**n)]), Or([And(is_blocking_point(nodes[i1], nodes[i2],time), local_ratify_prepared(nodes[i1], nodes[i2], x,time)) for i2 in range(n)])) == accept_prepared(nodes[i1], x,time) for i1 in range(n) for x in range(m)],
    # # (6) 原: Implies(well_behave(nodes[i1]), accept(nodes[i1], x) == local_accept_nomi(nodes[i2], nodes[i1], x))
    #     改: well_behave(..., time), accept_prepared(..., time), local_accept_prepared(..., time)
    [
        Implies(
            well_behave(nodes[i1], time),
            accept_prepared(nodes[i1], Ballot.cons(x, m), time)
            == local_accept_prepared(nodes[i2], nodes[i1], x, time)
        )
        for i1 in range(n)
        for i2 in range(n)
        for x in range(m)
    ],

    # (7) 原: Implies(well_behave(nodes[i1]), Or([... is_quorum(nsets[j]), ... local_accept_nomi(...)]) == confirm(nodes[i1], x))
    #     改: well_behave(..., time), is_quorum(..., time), local_accept_prepared(..., time), confirm_prepared(..., time)
    [
        Implies(
            well_behave(nodes[i1], time),
            Or([
                And(
                    Extract(i1, i1, nsets[j]) == 1,
                    is_quorum(nsets[j], time),
                    And([
                        Implies(
                            Extract(i2, i2, nsets[j]) == 1,
                            local_accept_prepared(nodes[i1], nodes[i2], x, time)
                        )
                        for i2 in range(n)
                    ])
                )
                for j in range(2**n)
            ]) == confirm_prepared(nodes[i1], Ballot.cons(x, m), time)
        )
        for i1 in range(n)
        for x in range(m)
    ],
]


In [13]:
 ballot = Const('ballot', Ballot)

In [14]:
CS = [
    # 第一条约束：p 的约束
    [
        Or(
            p(node_state(nodes[i1], time)) == ballot_nil,
            And(
                accept_prepared(nodes[i1], p(node_state(nodes[i1], time)), time),
                And([
                    Implies(
                        accept_prepared(nodes[i1], ballots[i_b], time),
                        Ballot.n(ballots[i_b]) <= Ballot.n(p(node_state(nodes[i1], time)))
                    )
                    for i_b in range(m)
                ])
            )
        )
        for i1 in range(n)
    ],


    [
        Or(
            p1(node_state(nodes[i1], time)) == ballot_nil,
            And(
                accept_prepared(nodes[i1], p1(node_state(nodes[i1], time)), time),
                Ballot.x(p(node_state(nodes[i1], time))) != Ballot.x(p1(node_state(nodes[i1], time))),
                And([
                    Implies(
                        accept_prepared(nodes[i1], ballots[i_b], time),
                        Ballot.n(ballots[i_b]) < Ballot.n(p(node_state(nodes[i1], time)))
                    )
                    for i_b in range(m)
                ])
            )
        )
        for i1 in range(n)
    ],


            # 第三条约束：h 的约束（phi=0 且 c=nil 时）
    [
        Implies(
            And(
                phi(node_state(nodes[i1], time)) == stage_consts[0],
                c(node_state(nodes[i1], time)) == ballot_nil
            ),
            Or(
                h(node_state(nodes[i1], time)) == ballot_nil,
                And(
                    confirm_prepared(nodes[i1], h(node_state(nodes[i1], time)), time),
                    And([
                        Implies(
                            confirm_prepared(nodes[i1], ballots[i_b], time),
                            Ballot.n(ballots[i_b]) <= Ballot.n(h(node_state(nodes[i1], time)))
                        )
                        for i_b in range(m)
                    ])
                )
            )
        )
        for i1 in range(n)
    ],


            # 第四条约束：h 和 c 的约束（phi=0 且 c≠nil 时）
    [
        Implies(
            And(
                phi(node_state(nodes[i1], time)) == stage_consts[0],
                c(node_state(nodes[i1], time)) != ballot_nil
            ),
            And(
                confirm_prepared(nodes[i1], c(node_state(nodes[i1], time)), time),
                confirm_prepared(nodes[i1], h(node_state(nodes[i1], time)), time),
                And([
                    Implies(
                        confirm_prepared(nodes[i1], ballots[i_b], time),
                        And(
                            Ballot.n(ballots[i_b]) <= Ballot.n(h(node_state(nodes[i1], time))),
                            Ballot.n(ballots[i_b]) <= Ballot.n(c(node_state(nodes[i1], time)))
                        )
                    )
                    for i_b in range(m)
                ])
            )
        )
        for i1 in range(n)
    ],


            # 第五条约束：phi=1 时的约束
    [
        Implies(
            phi(node_state(nodes[i1], time)) == stage_consts[1],
            And(
                accept_prepared(nodes[i1], c(node_state(nodes[i1], time)), time),
                accept_prepared(nodes[i1], h(node_state(nodes[i1], time)), time),
                And([
                    Implies(
                        accept_prepared(nodes[i1], ballots[i_b], time),
                        And(
                            Ballot.n(ballots[i_b]) <= Ballot.n(h(node_state(nodes[i1], time))),
                            Ballot.n(ballots[i_b]) <= Ballot.n(c(node_state(nodes[i1], time)))
                        )
                    )
                    for i_b in range(m)
                ])
            )
        )
        for i1 in range(n)
    ],


            # 第六条约束：phi=2 时的约束
    [
        Implies(
            phi(node_state(nodes[i1], time)) == stage_consts[2],
            And(
                confirm_prepared(nodes[i1], c(node_state(nodes[i1], time)), time),
                confirm_prepared(nodes[i1], h(node_state(nodes[i1], time)), time),
                And([
                    Implies(
                        confirm_prepared(nodes[i1], ballots[i_b], time),
                        And(
                            Ballot.n(ballots[i_b]) <= Ballot.n(h(node_state(nodes[i1], time))),
                            Ballot.n(ballots[i_b]) <= Ballot.n(c(node_state(nodes[i1], time)))
                        )
                    )
                    for i_b in range(m)
                ])
            )
        )
        for i1 in range(n)
    ],


            # 第七条约束：h.x 必须等于 z
    [
        Implies(
            h(node_state(nodes[i1], time)) != ballot_nil,
            Ballot.x(h(node_state(nodes[i1], time))) == z(node_state(nodes[i1], time))
        )
        for i1 in range(n)
    ]
]

In [15]:
C6 = [
    # 第一条约束：c.x == h.x == b.x
    [
        And(
            Ballot.x(c(node_state(nodes[i1], time))) == Ballot.x(h(node_state(nodes[i1], time))),
            Ballot.x(h(node_state(nodes[i1], time))) == Ballot.x(b(node_state(nodes[i1], time)))
        )
        for i1 in range(n)
    ],

    # 第二条约束：c.n <= h.n <= b.n
    [
        And(
            Ballot.n(c(node_state(nodes[i1], time))) <= Ballot.n(h(node_state(nodes[i1], time))),
            Ballot.n(h(node_state(nodes[i1], time))) <= Ballot.n(b(node_state(nodes[i1], time)))
        )
        for i1 in range(n)
    ]
]


In [16]:
accept_commit = Function('accept_commit', Node, Ballot, Time, BoolSort())
node2 = Const('node2', Node)

In [17]:
accept_commit_rules = [
    [
        (
            Or(
                # Quorum 路径
                Or([
                    And(
                        in_set(nodes[i1], nsets[j], time),
                        is_quorum(nsets[j], time),
                        And([
                            Implies(
                                in_set(nodes[i2], nsets[j], time),
                                local_accept_prepared(nodes[i1], nodes[i2], Ballot.x(ballot), time)
                            )
                            for i2 in range(n)
                        ])
                    )
                    for j in range(2**n)
                ]),
                # 阻塞点路径
                Or([
                    And(
                        is_blocking_point(nodes[i1], nodes[i2], time),
                        local_accept_prepared(nodes[i1], nodes[i2], Ballot.x(ballot), time)
                    )
                    for i2 in range(n)
                ])
            ) == accept_commit(nodes[i1], ballot, time)
        )
        for ballot in ballots
    ]
    for i1 in range(n)
]


In [18]:
confirm_commit = Function('confirm_commit', Node, Ballot, Time, BoolSort())
ballot2 = Const('ballot2', Ballot)

In [19]:
# 消除量词后的 confirm_commit_rules
confirm_commit_rules = [
    (
        And(
            # 条件 1: ballot 已经被一个 quorum 的节点 accept_commit
            Or(
                *[
                    And(
                        in_set(node1, nsets[j], time),    # 节点 node1 属于 nsets[j]
                        is_quorum(nsets[j], time),        # nsets[j] 是一个合法的 quorum
                        And(
                            *[
                                # 对所有 node2，若 node2 属于 nsets[j]，则 node2 接受了 ballot
                                Implies(in_set(node2, nsets[j], time), accept_commit(node2, ballot, time))
                                for node2 in nodes
                            ]
                        )
                    )
                    for j in range(2**n)  # 遍历所有可能的 nsets[j]
                ]
            ),
            # 条件 2: ballot 是当前阶段最高的 ballot
            And(
                *[
                    Implies(
                        And(
                            ballot2 != ballot,                     # ballot2 是另一个 ballot
                            Ballot.n(ballot2) > Ballot.n(ballot)     # ballot2 的编号更高
                        ),
                        Not(accept_commit(node1, ballot2, time))     # 没有更高的 ballot 被 accept_commit
                    )
                    for ballot2 in ballots  # 遍历所有可能的 ballot2
                ]
            )
        ) == confirm_commit(node1, ballot, time)  # 整体逻辑
    )
    for node1 in nodes
    for ballot in ballots
    for time in times
]



In [20]:
# 创建求解器
solver = Solver()

# 添加 accept_commit_rules 到求解器
solver.add(confirm_commit_rules)

# 检查是否可满足
if solver.check() == sat:
    print("是可满足的")
else:
    print("accept_commit_rules 不可满足")

是可满足的


In [21]:
# 定义 node_state_6、node_h1_6、node_c_6
node_state_6 = Function('node_state_6', Node, Time, BoolSort())
node_h1_6 = Function('node_h1_6', Node, Time, Ballot)
node_c_6 = Function('node_c_6', Node, Time, Ballot)

# 定义规则
node_state_6_rules = [
    node_state_6(node1, time) ==
    And(
        # 阶段为 CONFIRM
        phi(node_state(node1, time)) == stage_consts[1],
        # c 不为空
        c(node_state(node1, time)) != ballot_nil
    )
    for node1 in nodes
    for time in times
]

# 对于 node_h1_6_rules，由于有两个 ForAll 分别对应两个条件，
# 可以分两部分构造，再将它们合并成一个规则列表
node_h1_6_rules = [
    Implies(
        Ballot.n(c(node_state(node1, time))) > Ballot.n(h(node_state(node1, time))),
        node_h1_6(node1, time) == c(node_state(node1, time))
    )
    for node1 in nodes
    for time in times
] + [
    Implies(
        Ballot.n(c(node_state(node1, time))) <= Ballot.n(h(node_state(node1, time))),
        node_h1_6(node1, time) == h(node_state(node1, time))
    )
    for node1 in nodes
    for time in times
]

# 消除 node_c_6_rules 中的 ForAll，转换为对所有 (node1, time) 的等式
node_c_6_rules = [
    node_c_6(node1, time) == c(node_state(node1, time))
    for node1 in nodes
    for time in times
]
# 创建求解器
solver = Solver()

# 添加规则到求解器
solver.add(node_state_6_rules)
solver.add(node_h1_6_rules)
solver.add(node_c_6_rules)

# 检查是否可满足
if solver.check() == sat:
    print("规则是可满足的")
else:
    print("规则不可满足")


规则是可满足的


In [22]:
# 第一条约束：t 取值范围为 1 到 T-2，避免 time_consts[t+1] 越界
C2_pre_1 = [
    And([
        Implies(
            phi(node_state(node, time_consts[t-1])) == stage_consts[0],
            Implies(
                # 消除 Exists: 对所有 ballot 构造 Or 语句
                Or([
                    And(
                        Not(accept_prepared(node, ballot, time_consts[t-1])),
                        accept_prepared(node, ballot, time_consts[t])
                    )
                    for ballot in ballots
                ]),
                c(node_state(node, time_consts[t+1])) == ballot_nil
            )
        )
        for node in nodes
    ])
    for t in range(1, T-1)
]

# 第二条约束：t 取值范围为 1 到 T-1
C2_pre_2 = [
    And([
        Implies(
            phi(node_state(node, time_consts[t-1])) == stage_consts[0],
            Implies(
                And(
                    c(node_state(node, time_consts[t-1])) == ballot_nil,
                    Ballot.n(b(node_state(node, time_consts[t-1]))) <= Ballot.n(h(node_state(node, time_consts[t]))),
                    Not(
                        And(
                            Ballot.n(p(node_state(node, time_consts[t-1]))) >= Ballot.n(h(node_state(node, time_consts[t]))),
                            Ballot.x(p(node_state(node, time_consts[t-1]))) != Ballot.x(h(node_state(node, time_consts[t])))
                        )
                    ),
                    Not(
                        And(
                            Ballot.n(p1(node_state(node, time_consts[t-1]))) >= Ballot.n(h(node_state(node, time_consts[t]))),
                            Ballot.x(p1(node_state(node, time_consts[t-1]))) != Ballot.x(h(node_state(node, time_consts[t])))
                        )
                    )
                ),
                And(
                    Ballot.n(c(node_state(node, time_consts[t]))) >= Ballot.n(b(node_state(node, time_consts[t-1]))),
                    # 内部 ForAll(ballot, …) 消除为对所有 ballot 的 And 语句
                    And([
                        Implies(
                            And(
                                Ballot.n(ballot) >= Ballot.n(b(node_state(node, time_consts[t-1]))),
                                Ballot.n(ballot) <= Ballot.n(h(node_state(node, time_consts[t]))),
                                Ballot.x(ballot) == Ballot.x(h(node_state(node, time_consts[t])))
                            ),
                            Ballot.n(ballot) <= Ballot.n(c(node_state(node, time_consts[t])))
                        )
                        for ballot in ballots
                    ])
                )
            )
        )
        for node in nodes
    ])
    for t in range(1, T)
]

# 第三条约束：t 取值范围为 1 到 T-1
C2_pre_3 = [
    And([
        Implies(
            phi(node_state(node, time_consts[t-1])) == stage_consts[0],
            Implies(
                # Exists(ballot, accept_commit(...)) 消除为对所有 ballot 的 Or 语句
                Or([accept_commit(node, ballot, time_consts[t-1]) for ballot in ballots]),
                And(
                    phi(node_state(node, time_consts[t])) == stage_consts[1],
                    Implies(
                        Not(
                            And(
                                Ballot.n(h(node_state(node, time_consts[t]))) <= Ballot.n(b(node_state(node, time_consts[t-1]))),
                                Ballot.x(h(node_state(node, time_consts[t]))) == Ballot.x(b(node_state(node, time_consts[t-1])))
                            )
                        ),
                        And(
                            Ballot.x(b(node_state(node, time_consts[t]))) == Ballot.x(h(node_state(node, time_consts[t]))),
                            node_state_6(node, time_consts[t])
                        )
                    )
                )
            )
        )
        for node in nodes
    ])
    for t in range(1, T)
]

# 将所有约束组合在一起（如果需要）
C2_pre = [C2_pre_1, C2_pre_2, C2_pre_3]


In [23]:
# 第一组约束：t 的范围为 1 到 T-2，避免 time_consts[t+1] 越界
C2_con_1 = [
    And([
        Implies(
            phi(node_state(node, time_consts[t-1])) == stage_consts[1],
            Implies(
                Or(
                    # 消除 Exists：对所有 ballot 取 Or
                    Or([
                        And(
                            Not(accept_commit(node, ballot, time_consts[t-1])),
                            accept_commit(node, ballot, time_consts[t])
                        )
                        for ballot in ballots
                    ]),
                    node_state_6(node, time_consts[t])
                ),
                And(
                    Implies(
                        Ballot.n(node_h1_6(node, time_consts[t])) > Ballot.n(h(node_state(node, time_consts[t]))),
                        h(node_state(node, time_consts[t+1])) == node_h1_6(node, time_consts[t])
                    ),
                    c(node_state(node, time_consts[t+1])) == node_c_6(node, time_consts[t])
                )
            )
        )
        for node in nodes
    ])
    for t in range(1, T-1)
]

# 第二组约束：t 的范围为 1 到 T-1
C2_con_2 = [
    And([
        Implies(
            phi(node_state(node, time_consts[t-1])) == stage_consts[1],
            Implies(
                # 消除 Exists：对所有 ballot 取 Or
                Or([
                    confirm_commit(node, ballot, time_consts[t-1])
                    for ballot in ballots
                ]),
                phi(node_state(node, time_consts[t])) == stage_consts[2]
            )
        )
        for node in nodes
    ])
    for t in range(1, T)
]

# 将两组约束组合到一起
C2_con = [C2_con_1, C2_con_2]



In [24]:
C2_ext = [
    [
        And([
            Implies(
                phi(node_state(node, time_consts[t-1])) == stage_consts[2],
                node_state(node, time_consts[t-1]) == node_state(node, time_consts[t])
            )
            for node in nodes
        ])
        for t in range(1, T)
    ]
]


In [25]:
C2_raise_b = [
    [
        ForAll([node1], 
            Implies(
                Ballot.n(b(node_state(node1, time_consts[t-1]))) < Ballot.n(h(node_state(node1, time_consts[t-1]))),  # 使用 b(node_state(...)) 和 h(node_state(...)) 获取 b 和 h
                node_state_6(node1, time_consts[t])  
            )
        )
        for t in range(1, T)  
    ],
]

In [26]:
qslice_matrix = (
    (1,),
    (1,),
    (1,),
    (1,),
    (1,),
    (1,),
)
# 提前指定提名结构
init_pre_matrix = (
    (1,),
    (1,),
    (1,),
    (1,),
    (1,),
    (0,),
)
# 提前制定好坏节点
well_behave_vec = (1, 1,1,1,1, 0,)

In [27]:
C3 = [in_qslice(node_consts[i], node_consts[j],time) if qslice_matrix[i][0] else Not(in_qslice(node_consts[i], node_consts[j],time)) for i in range(n) for j in range(n)for time in times]


In [28]:
C4 = [init_prepared(node_consts[i], x,time) if init_pre_matrix[i][j] else Not(init_prepared(node_consts[i], x,time)) for i in range(n) for j in range(m)for x in range(m)for time in times]


In [29]:
C5 = [well_behave(node_consts[i],time) if well_behave_vec[i] else Not(well_behave(node_consts[i],time)) for i in range(n)for time in times]


In [30]:
S1 = [Implies(And(is_intact(node_consts[i],time), is_intact(node_consts[j],time)), confirm_commit(node_consts[i], ballots[x],time) == confirm_commit(node_consts[j], ballots[x],time)) for i in range(n) for j in range(n) for x in range(m) for time in times]

In [31]:
# n=3
s = Solver()
s.add(flatten(Defi2)+ flatten(C1)+flatten(C6)+flatten(CS)+flatten(accept_commit_rules)+confirm_commit_rules
+node_state_6_rules+node_c_6_rules+node_h1_6_rules+flatten(C2_pre)+flatten(C2_con)+flatten(C2_ext)+flatten(Init)+ [Not(s) for s in S1]
)
s.check()

unsat

In [ ]:
t_final = times[-1]
finalization_condition = Or([
    And([Implies(is_intact(node_consts[i], t_final), 
                 confirm_commit(node_consts[i], ballots[x], t_final))
         for i in range(n)])
    for x in range(m)
])
s = Solver()
s.add(flatten(Defi2) + flatten(C1) + flatten(C6) + flatten(CS) +
      flatten(accept_commit_rules) + confirm_commit_rules +
      node_state_6_rules + node_c_6_rules + node_h1_6_rules +
      flatten(C2_pre) + flatten(C2_con) + flatten(C2_ext) + flatten(Init) +
      [Not(s) for s in S1]
)
s.add(Not(finalization_condition))  # 检查反例

if s.check() == unsat:
    print("在最后一个时刻，该值已经被最终外化")
else:
    print("在最后一个时刻，该值未被最终外化")


在最后一个时刻，该值已经被最终外化


In [33]:
# CS = [
#     # 第一条约束：p 的约束
#         ForAll([node1, time], 
#             Or(
#                 p(node_state(node1, time)) == ballot_nil, 
#                 And(
#                     accept_prepared(node1, p(node_state(node1, time)), time), 
#                     ForAll([ballot], 
#                         Implies(
#                             accept_prepared(node1, ballot, time), 
#                             Ballot.n(ballot) <= Ballot.n(p(node_state(node1, time)))  # 使用 Ballot.n(ballot)
#                         )
#                     )
#                 )
#             )
#         )

#         # 第二条约束：p1 的约束
#         ForAll([node1, time], 
#             Or(
#                 p1(node_state(node1, time)) == ballot_nil, 
#                 And(
#                     accept_prepared(node1, p1(node_state(node1, time)), time), 
#                     Ballot.x(p(node_state(node1, time))) != Ballot.x(p1(node_state(node1, time))),  # 使用 Ballot.x(ballot)
#                     ForAll([ballot], 
#                         Implies(
#                             accept_prepared(node1, ballot, time), 
#                             Ballot.n(ballot) < Ballot.n(p(node_state(node1, time)))  # 使用 Ballot.n(ballot)
#                         )
#                     )
#                 )
#             )
#         ),

#         # 第三条约束：h 的约束（phi=0 且 c=nil 时）
#         ForAll([node1, time], 
#             Implies(
#                 And(
#                     phi(node_state(node1, time)) == stage_consts[0], 
#                     c(node_state(node1, time)) == ballot_nil
#                 ), 
#                 Or(
#                     h(node_state(node1, time)) == ballot_nil, 
#                     And(
#                         confirm_prepared(node1, h(node_state(node1, time)), time), 
#                         ForAll([ballot], 
#                             Implies(
#                                 confirm_prepared(node1, ballot, time), 
#                                 Ballot.n(ballot) <= Ballot.n(h(node_state(node1, time)))  # 使用 Ballot.n(ballot)
#                             )
#                         )
#                     )
#                 )
#             )
#         ),

#         # 第四条约束：h 和 c 的约束（phi=0 且 c≠nil 时）
#         ForAll([node1, time], 
#             Implies(
#                 And(
#                     phi(node_state(node1, time)) == stage_consts[0], 
#                     c(node_state(node1, time)) != ballot_nil
#                 ), 
#                 And(
#                     confirm_prepared(node1, c(node_state(node1, time)), time), 
#                     confirm_prepared(node1, h(node_state(node1, time)), time), 
#                     ForAll([ballot], 
#                         Implies(
#                             confirm_prepared(node1, ballot, time), 
#                             And(
#                                 Ballot.n(ballot) <= Ballot.n(h(node_state(node1, time))),  # 使用 Ballot.n(ballot)
#                                 Ballot.n(ballot) <= Ballot.n(c(node_state(node1, time)))   # 使用 Ballot.n(ballot)
#                             )
#                         )
#                     )
#                 )
#             )
#         ),

#         # 第五条约束：phi=1 时的约束
#         ForAll([node1, time], 
#             Implies(
#                 phi(node_state(node1, time)) == stage_consts[1], 
#                 And(
#                     accept_prepared(node1, c(node_state(node1, time)), time), 
#                     accept_prepared(node1, h(node_state(node1, time)), time), 
#                     ForAll([ballot], 
#                         Implies(
#                             accept_prepared(node1, ballot, time), 
#                             And(
#                                 Ballot.n(ballot) <= Ballot.n(h(node_state(node1, time))),  # 使用 Ballot.n(ballot)
#                                 Ballot.n(ballot) <= Ballot.n(c(node_state(node1, time)))   # 使用 Ballot.n(ballot)
#                             )
#                         )
#                     )
#                 )
#             )
#         ),

#         # 第六条约束：phi=2 时的约束
#         ForAll([node1, time], 
#             Implies(
#                 phi(node_state(node1, time)) == stage_consts[2], 
#                 And(
#                     confirm_prepared(node1, c(node_state(node1, time)), time), 
#                     confirm_prepared(node1, h(node_state(node1, time)), time), 
#                     ForAll([ballot], 
#                         Implies(
#                             confirm_prepared(node1, ballot, time), 
#                             And(
#                                 Ballot.n(ballot) <= Ballot.n(h(node_state(node1, time))),  # 使用 Ballot.n(ballot)
#                                 Ballot.n(ballot) <= Ballot.n(c(node_state(node1, time)))   # 使用 Ballot.n(ballot)
#                             )
#                         )
#                     )
#                 )
#             )
#         ),

#         # 第七条约束：h.x 必须等于 z
#         ForAll([node1, time], 
#             Implies(
#                 h(node_state(node1, time)) != ballot_nil, 
#                 Ballot.x(h(node_state(node1, time))) == z(node_state(node1, time))  # 使用 Ballot.x(ballot)
#             )
#         )
#     ]

In [34]:
# C6 = [
#     # 第一条约束：c.x == h.x == b.x
#     ForAll([node1, time], 
#         And(
#             Ballot.x(c(node_state(node1, time))) == Ballot.x(h(node_state(node1, time))),  # c.x == h.x
#             Ballot.x(h(node_state(node1, time))) == Ballot.x(b(node_state(node1, time)))    # h.x == b.x
#         )
#     ),

#     # 第二条约束：c.n <= h.n <= b.n
#     ForAll([node1, time], 
#         And(
#             Ballot.n(c(node_state(node1, time))) <= Ballot.n(h(node_state(node1, time))),  # c.n <= h.n
#             Ballot.n(h(node_state(node1, time))) <= Ballot.n(b(node_state(node1, time)))   # h.n <= b.n
#         )
#     )
# ]


In [35]:
# # 定义 accept_commit_rules
# accept_commit_rules = [
#     ForAll([node1, ballot, time], 
#         Or(
#             # Quorum 路径
#             Or([  # 使用 Or 遍历所有可能的 nsets[j]
#                 And(
#                     in_set(node1, nsets[j], time),  # 节点 node1 属于 nsets[j]
#                     is_quorum(nsets[j], time),  # nsets[j] 是一个合法的 quorum
#                     ForAll([node2], 
#                         Implies(
#                             in_set(node2, nsets[j], time),  # 节点 node2 属于 nsets[j]
#                             local_accept_prepared(node1, node2, Ballot.x(ballot), time)  # 节点 node1 本地观测到 node2 接受了 ballot 的提交
#                         )
#                     )
#                 )
#                 for j in range(2**n)  # 遍历所有可能的 nsets[j]
#             ]), 
#             # 阻塞点路径
#             Exists([node2], 
#                 And(
#                     is_blocking_point(node1, node2, time),  # 节点 node2 是 node1 的阻塞点
#                     local_accept_prepared(node1, node2, Ballot.x(ballot), time)  # 节点 node1 本地观测到 node2 接受了 ballot 的提交
#                 )
#             )
#         ) == accept_commit(node1, ballot, time)  # 整体逻辑
#     )
# ]

In [36]:
# confirm_commit_rules = [
#     ForAll([node1, ballot, time], 
#         And(
#             # 条件 1: ballot 已经被一个 quorum 的节点 accept_commit
#             Or([  # 使用 Or 遍历所有可能的 nsets[j]
#                 And(
#                     in_set(node1, nsets[j], time),  # 节点 node1 属于 nsets[j]
#                     is_quorum(nsets[j], time),  # nsets[j] 是一个合法的 quorum
#                     ForAll([node2], 
#                         Implies(
#                             in_set(node2, nsets[j], time),  # 节点 node2 属于 nsets[j]
#                             accept_commit(node2, ballot, time)  # 节点 node2 接受了 ballot
#                         )
#                     )
#                 )
#                 for j in range(2**n)  # 遍历所有可能的 nsets[j]
#             ]),
#             # 条件 2: ballot 是当前阶段最高的 ballot
#             ForAll([ballot2], 
#                 Implies(
#                     And(
#                         ballot2 != ballot,  # ballot2 是另一个 ballot
#                         Ballot.n(ballot2) > Ballot.n(ballot)  # ballot2 的编号更高
#                     ),
#                     Not(accept_commit(node1, ballot2, time))  # 没有更高的 ballot 被 accept_commit
#                 )
#             )
#         ) == confirm_commit(node1, ballot, time)  # 整体逻辑
#     )
# ]